# Projet A - Détecteur de Fake News Avancé

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Améliorer et déployer un détecteur de fake news

---

## Objectifs du projet

Dans ce projet, vous allez :
1. Utiliser un modèle pré-entraîné (DistilBERT) pour la classification
2. Comparer les performances avec votre Transformer from scratch
3. Créer une interface de démonstration
4. Analyser les erreurs et limites du modèle

## 0. Installation

In [ ]:
!pip install torch transformers datasets matplotlib numpy scikit-learn tqdm ipywidgets -q

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    pipeline,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

## 1. Chargement des données

In [ ]:
# Charger le dataset
dataset = load_dataset("liar", trust_remote_code=True)

# Simplifier les labels (Fake vs Real)
def simplify_label(example):
    example['binary_label'] = 0 if example['label'] < 3 else 1
    return example

dataset = dataset.map(simplify_label)
print(f"Train: {len(dataset['train'])} | Val: {len(dataset['validation'])} | Test: {len(dataset['test'])}")

---

## 2. Approche HuggingFace : DistilBERT

Utilisons un modèle pré-entraîné et comparons avec notre Transformer.

In [ ]:
# Charger tokenizer et modèle
model_name = "distilbert-base-uncased"

tokenizer_bert = AutoTokenizer.from_pretrained(model_name)
model_bert = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
).to(device)

print(f"Modèle chargé: {model_name}")
print(f"Paramètres: {sum(p.numel() for p in model_bert.parameters()):,}")

In [ ]:
# Préparer les données pour HuggingFace
def tokenize_function(examples):
    return tokenizer_bert(
        examples['statement'], 
        padding='max_length', 
        truncation=True,
        max_length=128
    )

# Tokenizer le dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Renommer la colonne label
tokenized_dataset = tokenized_dataset.rename_column('binary_label', 'labels')

# Garder seulement les colonnes nécessaires
tokenized_dataset = tokenized_dataset.remove_columns(
    [c for c in tokenized_dataset['train'].column_names 
     if c not in ['input_ids', 'attention_mask', 'labels']]
)

tokenized_dataset.set_format('torch')

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Métriques
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}

# Trainer
trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics,
)

In [ ]:
# Entraîner
print("Entraînement de DistilBERT...")
trainer.train()

In [ ]:
# Évaluation
results = trainer.evaluate(tokenized_dataset['test'])
print(f"\nRésultats sur le test set:")
print(f"Accuracy: {results['eval_accuracy']:.4f}")

---

## 3. Pipeline complet avec traduction

In [ ]:
# Créer les pipelines
translator = pipeline("translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en")
classifier = pipeline("text-classification", model=model_bert, tokenizer=tokenizer_bert, device=0 if torch.cuda.is_available() else -1)

In [ ]:
class FakeNewsDetectorPro:
    """
    Détecteur de Fake News avancé avec support multilingue.
    """
    
    def __init__(self, classifier, translator=None):
        self.classifier = classifier
        self.translator = translator
        self.label_map = {'LABEL_0': 'FAKE', 'LABEL_1': 'REAL'}
    
    def predict(self, text, source_lang='en'):
        """
        Prédit si un texte est une fake news.
        
        Args:
            text: Texte à analyser
            source_lang: Langue du texte ('en' ou 'fr')
        """
        # Traduction si nécessaire
        if source_lang == 'fr' and self.translator:
            translation = self.translator(text, max_length=200)[0]['translation_text']
            text_to_classify = translation
        else:
            translation = None
            text_to_classify = text
        
        # Classification
        result = self.classifier(text_to_classify)[0]
        
        return {
            'original_text': text,
            'translation': translation,
            'label': self.label_map.get(result['label'], result['label']),
            'confidence': result['score'],
            'source_lang': source_lang
        }
    
    def analyze_batch(self, texts, source_lang='en'):
        """Analyse un lot de textes."""
        results = []
        for text in texts:
            results.append(self.predict(text, source_lang))
        return results

In [ ]:
# Créer le détecteur
detector = FakeNewsDetectorPro(classifier, translator)

# Tests
tests_fr = [
    "Le gouvernement a annoncé une réforme des retraites.",
    "Les vaccins contiennent des puces 5G pour nous contrôler.",
    "L'économie française a progressé de 0.5% ce trimestre.",
    "Des pyramides ont été découvertes sur Mars par la NASA.",
]

print("=" * 70)
print("DÉTECTION DE FAKE NEWS (Français)")
print("=" * 70)

for text in tests_fr:
    result = detector.predict(text, source_lang='fr')
    emoji = "❌" if result['label'] == 'FAKE' else "✅"
    print(f"\n{emoji} {result['label']} ({result['confidence']:.1%})")
    print(f"   FR: {text}")
    print(f"   EN: {result['translation']}")

---

## 4. Analyse des erreurs

In [ ]:
# Analyser les erreurs sur le test set
test_data = dataset['test']

# Échantillon pour analyse
sample_size = 200
indices = np.random.choice(len(test_data), sample_size, replace=False)

errors = []
correct = []

for idx in tqdm(indices, desc="Analyse"):
    item = test_data[int(idx)]
    text = item['statement']
    true_label = item['binary_label']
    
    result = classifier(text)[0]
    pred_label = 0 if result['label'] == 'LABEL_0' else 1
    
    record = {
        'text': text,
        'true_label': 'FAKE' if true_label == 0 else 'REAL',
        'pred_label': 'FAKE' if pred_label == 0 else 'REAL',
        'confidence': result['score']
    }
    
    if pred_label != true_label:
        errors.append(record)
    else:
        correct.append(record)

print(f"\nCorrects: {len(correct)} | Erreurs: {len(errors)}")
print(f"Accuracy échantillon: {len(correct)/sample_size:.1%}")

In [ ]:
# Examiner quelques erreurs
print("\n" + "="*70)
print("EXEMPLES D'ERREURS")
print("="*70)

for i, err in enumerate(errors[:5]):
    print(f"\n[{i+1}] Texte: {err['text'][:100]}...")
    print(f"    Vérité: {err['true_label']} | Prédiction: {err['pred_label']} ({err['confidence']:.1%})")

---

## 5. Interface de démonstration

In [ ]:
# Interface simple avec input
def demo_interactive():
    print("\n" + "="*50)
    print("DÉTECTEUR DE FAKE NEWS - Démo Interactive")
    print("="*50)
    print("Entrez un texte (en français ou anglais)")
    print("Tapez 'quit' pour quitter")
    print("="*50)
    
    while True:
        text = input("\nTexte: ")
        if text.lower() == 'quit':
            break
        
        # Détection automatique de la langue (simple)
        fr_words = ['le', 'la', 'les', 'de', 'du', 'des', 'est', 'sont', 'a', 'ont']
        is_french = any(w in text.lower().split() for w in fr_words)
        lang = 'fr' if is_french else 'en'
        
        result = detector.predict(text, source_lang=lang)
        
        emoji = "❌ FAKE" if result['label'] == 'FAKE' else "✅ REAL"
        print(f"\nRésultat: {emoji}")
        print(f"Confiance: {result['confidence']:.1%}")
        if result['translation']:
            print(f"Traduction: {result['translation']}")

# Décommenter pour tester
# demo_interactive()

---

## 6. Exercices

### Exercice 1 : Améliorer le modèle
Essayez d'autres modèles pré-entraînés et comparez les performances.

### Exercice 2 : Analyser les biais
Le modèle a-t-il des biais ? Testez avec différents sujets politiques.

### Exercice 3 : Créer un rapport
Documentez vos résultats et observations.

In [ ]:
# Espace pour vos expérimentations

# Exercice 1 : Essayer un autre modèle
# other_model = "bert-base-uncased"
# ...

# Exercice 2 : Tests de biais
test_sujets = [
    # Politique
    "The president announced new economic measures.",
    "The opposition criticized the government's policies.",
    
    # Santé
    "Scientists discovered a new treatment for cancer.",
    "Natural remedies can cure all diseases.",
    
    # Technologie
    "Apple released a new iPhone model.",
    "5G towers cause coronavirus.",
]

print("Tests par sujet:")
for text in test_sujets:
    result = detector.predict(text, source_lang='en')
    print(f"  {result['label']:4} ({result['confidence']:.0%}): {text[:50]}...")

---

## 7. Conclusion

### Ce que vous avez appris

1. Utiliser des modèles pré-entraînés (HuggingFace)
2. Fine-tuner pour une tâche spécifique
3. Créer un pipeline multilingue
4. Analyser les erreurs et biais

### Limitations du système

- Dépend de la qualité de la traduction
- Biais du dataset d'entraînement (anglais, US-centric)
- Ne vérifie pas les faits, détecte des patterns
- Confiance ≠ Vérité

### Pour aller plus loin

- Utiliser des modèles multilingues (mBERT, XLM-RoBERTa)
- Intégrer des sources de vérification de faits
- Combiner avec analyse de sources